Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [158]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [159]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [160]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px
import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score

In [161]:
# change created to data time format for subsetting by month
df['created'] = pd.to_datetime(df['created'])

In [163]:
# create column with month value
df['month'] = df['created'].map(lambda x: x.month)

In [164]:
# check
df['month'].value_counts()

6    16973
4    16217
5    15627
Name: month, dtype: int64

In [224]:
# create column for 'allows pets'
df['pets'] = df['cats_allowed'] + df['dogs_allowed']

In [167]:
# remap the values to with 1=allows pets, 0=doesnt allow pets
df['pets'] = df['pets'].mask(df['pets']==1,df['pets'].replace(1,0))
df['pets'] = df['pets'].mask(df['pets']==2,df['pets'].replace(2,1))

In [238]:
# create column that show the total number of rooms.
df['num_rooms'] = df['bedrooms'] + df['bathrooms']

In [200]:
# split data
train_months = [4,5]
train = df[df['month'].isin(train_months)]
test = df[df['month'] == 6]

In [225]:
# subset training and test data

model =LinearRegression()

features = ['bedrooms', 'bathrooms', 'num_rooms','pets']
target = ['price']

X_train = train[features]
X_test  = test[features]
y_train = train[target]
y_test  = test[target]

In [235]:
# create model 

model.fit(X_train, y_train)

y_pred = model.predict(X_train)

mae = mean_absolute_error(y_train,y_pred)
mse = mean_squared_error(y_train,y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_train,y_pred)

print(f'Training Set errors-\nmean absolute error: {mae}\nmean squared error: {mse}\nRMSE: {rmse}\nr2: {r2}')

Training Set errors-
mean absolute error: 814.2056127613206
mean squared error: 1510428.2362635205
RMSE: 1228.994807256532
r2: 0.5135541031284678


In [236]:
model.fit(X_test,y_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [237]:
y_pred = model.predict(X_test)
mean_absolute_error(y_test,y_pred)

823.8060492702045